# Introduction
State notebook purpose here

### Imports
Import libraries and write settings here.

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30
pd.options.display.float_format = '{:,.4f}'.format

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import seaborn as sns
#import plotly.plotly as py
#import plotly.graph_objs as go
#from plotly.offline import iplot, init_notebook_mode
#init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

/home/luca/anaconda3/envs/tensorflow/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:558: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis


/home/luca/anaconda3/envs/tensorflow/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:531: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis




## Custom imports

In [1]:
import sys
# code_path = "/home/luca/Desktop/MLaaS4HEP/src/python/"
# sys.path.append(code_path)

import json
from MLaaS4HEP.reader import JSONReader

# Analysis/Modeling
Do work here

## Try custom import

In [3]:
import json
with open("rucio-opint.web.cern.ch.json", 'r') as f:
    raw_data = json.load(f)

tot_errors = raw_data["count"]

In [4]:
print("Total number of errors:", tot_errors)

Total number of errors: 1819


In [5]:
errors = pd.DataFrame(raw_data["results"]).set_index("id")

errors.head()

,amount,category,dst_site,last_modified,message,src_site,status,type
id,,,,,,,,
1,178,9,BNL-ATLAS,2019-08-06T14:07:04.412401+02:00,SOURCE [70] globus_xio: Unable to connect to a...,BU_ATLAS_Tier2,New,transfer-failure
2,3288,10,BU_ATLAS_Tier2,2019-08-06T14:07:04.595574+02:00,TRANSFER [70] TRANSFER globus_xio: Unable to ...,BNL-ATLAS,New,transfer-failure
3,3288,10,BU_ATLAS_Tier2,2019-08-06T14:07:04.692554+02:00,TRANSFER [70] TRANSFER globus_xio: Unable to ...,INFN-NAPOLI-ATLAS,New,transfer-failure
4,227,11,CERN-PROD,2019-08-06T14:07:04.803866+02:00,SOURCE [70] globus_xio: Unable to connect to a...,BU_ATLAS_Tier2,New,transfer-failure
5,130,11,TOKYO-LCG2,2019-08-06T14:07:04.891438+02:00,SOURCE [70] globus_xio: Unable to connect to a...,BU_ATLAS_Tier2,New,transfer-failure


## Try simple clustering approach

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(errors.message)

In [7]:
true_k = 5

# Define the model with initialization
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)

# Fit the model
model.fit(X)

# Get the resulting centroids
order_centroids = model.cluster_centers_.argsort()[:, ::-1]

# Get features
terms = vectorizer.get_feature_names()

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=5, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [11]:
df = pd.DataFrame(order_centroids, columns=terms , index=['Group_0', 'Group_1', 'Group_2', 'Group_3', 'Group_4']) 
df.T.head(100)

,Group_0,Group_1,Group_2,Group_3,Group_4
05,46,199,69,60,106
100,193,172,89,154,155
10025,72,27,71,67,73
110,51,18,75,133,62
1264,19,65,50,205,123
1458,107,179,208,40,145
19,65,59,109,111,58
2001,179,55,95,162,68
201,114,194,137,33,76
21,20,153,90,129,75


In [10]:
model.n_iter_

3

## Try MLaaS4HEP

In [59]:
read = JSONReader("rucio-opint.web.cern.ch.json", label="first100", verbose=0)

for data in read.next():
    errors = data

In [49]:
for data in read.next():
    print((data[0][3][102]))

{'id': 1, 'message': 'SOURCE [70] globus_xio: Unable to connect to atlas-gridftp.bu.edu:2811 globus_xio: globus_libc_getaddrinfo failed. globus_common: Temporary failure in name resolution ', 'src_site': 'BU_ATLAS_Tier2', 'dst_site': 'BNL-ATLAS', 'category': 9, 'amount': 178, 'type': 'transfer-failure', 'status': 'New', 'last_modified': '2019-08-06T14:07:04.412401+02:00'}


In [66]:
errors[0][2]

'r'

In [14]:
data = read.next()["results"]

TypeError: 'generator' object is not subscriptable

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here